<h1><center> Régression Logistique </center></h1>


## 1. Importez les bibliothéques necessaires 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## 2. Importez le jeu de données 'weatherAUS.csv'

Ce jeu de données contient des observations météorologiques quotidiennes de nombreuses stations météorologiques australiennes.

Pour plus d'information sur le jeu de données: https://www.kaggle.com/jsphyg/weather-dataset-rattle-package

In [3]:
df = pd.read_csv('data_logit/weatherAUS.csv')

## 3. Exploration des données


### Dimension de du jeu de données

In [4]:

df.shape

(142193, 24)

### Prévisulaiser le jeu de données

In [5]:

df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RISK_MM,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,0.0,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,0.0,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,0.0,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,1.0,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,0.2,No


### Afficher la liste des labels des colonnes 


In [6]:
col_names = df.columns

list(col_names)

['Date',
 'Location',
 'MinTemp',
 'MaxTemp',
 'Rainfall',
 'Evaporation',
 'Sunshine',
 'WindGustDir',
 'WindGustSpeed',
 'WindDir9am',
 'WindDir3pm',
 'WindSpeed9am',
 'WindSpeed3pm',
 'Humidity9am',
 'Humidity3pm',
 'Pressure9am',
 'Pressure3pm',
 'Cloud9am',
 'Cloud3pm',
 'Temp9am',
 'Temp3pm',
 'RainToday',
 'RISK_MM',
 'RainTomorrow']

### Supprimer la colonne 'RISK_MM' du jeu de données

In [7]:
df.drop(['RISK_MM'], axis=1, inplace=True)


### Vérifier si la colonne est supprimée

In [8]:
df.head()
# df.shape

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


### Afficher un summary du jeu de données

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142193 entries, 0 to 142192
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           142193 non-null  object 
 1   Location       142193 non-null  object 
 2   MinTemp        141556 non-null  float64
 3   MaxTemp        141871 non-null  float64
 4   Rainfall       140787 non-null  float64
 5   Evaporation    81350 non-null   float64
 6   Sunshine       74377 non-null   float64
 7   WindGustDir    132863 non-null  object 
 8   WindGustSpeed  132923 non-null  float64
 9   WindDir9am     132180 non-null  object 
 10  WindDir3pm     138415 non-null  object 
 11  WindSpeed9am   140845 non-null  float64
 12  WindSpeed3pm   139563 non-null  float64
 13  Humidity9am    140419 non-null  float64
 14  Humidity3pm    138583 non-null  float64
 15  Pressure9am    128179 non-null  float64
 16  Pressure3pm    128212 non-null  float64
 17  Cloud9am       88536 non-null

### Afficher la liste des noms des variables qualitatives 

In [10]:
quali=[]
for var in df.columns:
    if df[var].dtype=='O':
        quali.append(var)
quali

['Date',
 'Location',
 'WindGustDir',
 'WindDir9am',
 'WindDir3pm',
 'RainToday',
 'RainTomorrow']

In [11]:
categorical = [var for var in df.columns if df[var].dtype=='O']

categorical

['Date',
 'Location',
 'WindGustDir',
 'WindDir9am',
 'WindDir3pm',
 'RainToday',
 'RainTomorrow']

In [12]:
categorical = [var for var in df.columns if df[var].dtype=='O']

print('Il y a {} variables qualitatives \n'.format(len(categorical)))

print('Les variables qualitatives sont :', categorical)

Il y a 7 variables qualitatives 

Les variables qualitatives sont : ['Date', 'Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm', 'RainToday', 'RainTomorrow']


### Afficher le contenu de ces variables qualitatives

In [13]:

df[categorical].head()

,Date,Location,WindGustDir,WindDir9am,WindDir3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,W,W,WNW,No,No
1,2008-12-02,Albury,WNW,NNW,WSW,No,No
2,2008-12-03,Albury,WSW,W,WSW,No,No
3,2008-12-04,Albury,NE,SE,E,No,No
4,2008-12-05,Albury,W,ENE,NW,No,No


### Checker le nombre des valeurs manquantes dans chaque variable qualitative

In [14]:


df[categorical].isnull().sum()

Date                0
Location            0
WindGustDir      9330
WindDir9am      10013
WindDir3pm       3778
RainToday        1406
RainTomorrow        0
dtype: int64

### Afficher les variables qualitatives qui contiennent des valeurs manquantes

In [38]:
for var in categorical:
    
    if df[var].isnull().sum()!=0:
        print(var)
    

WindGustDir
WindDir9am
WindDir3pm
RainToday


In [39]:

cat1 = [var for var in categorical if df[var].isnull().sum()!=0]

print(cat1)

['WindGustDir', 'WindDir9am', 'WindDir3pm', 'RainToday']


### Afficher la fréquence des valeurs des variables qualitatives

In [13]:
for var in categorical: 
    print("========="+str(var)+"=========")
    print(df[var].value_counts())


=========Date=========
2014-01-16    49
2016-08-31    49
2014-04-21    49
2014-02-03    49
2013-04-30    49
              ..
2007-12-24     1
2007-12-12     1
2007-12-23     1
2008-01-10     1
2007-11-24     1
Name: Date, Length: 3436, dtype: int64
=========Location=========
Canberra            3418
Sydney              3337
Perth               3193
Darwin              3192
Hobart              3188
Brisbane            3161
Adelaide            3090
Bendigo             3034
Townsville          3033
AliceSprings        3031
MountGambier        3030
Launceston          3028
Ballarat            3028
Albany              3016
Albury              3011
MelbourneAirport    3009
PerthAirport        3009
Mildura             3007
SydneyAirport       3005
Nuriootpa           3002
Sale                3000
Watsonia            2999
Tuggeranong         2998
Portland            2996
Woomera             2990
Cobar               2988
Cairns              2988
Wollongong          2983
GoldCoast           2980

### Nombre de labels: cardinalité

Le nombre de labels dans une variable qualitative est appelé la cardinalité. Un nombre élevé de labels dans une variable est appelé cardinalité élevée. Une cardinalité élevée peut poser de sérieux problèmes dans le modèle d'apprentissage automatique. Donc, Afficher la cardinalité de chaque variable qualitative.

In [42]:
for var in categorical:
    
#     print(df[var].unique())
    
    print(var, ' contient ', len(df[var].unique()), ' labels')

['2008-12-01' '2008-12-02' '2008-12-03' ... '2008-01-29' '2008-01-30'
 '2008-01-31']
Date  contient  3436  labels
['Albury' 'BadgerysCreek' 'Cobar' 'CoffsHarbour' 'Moree' 'Newcastle'
 'NorahHead' 'NorfolkIsland' 'Penrith' 'Richmond' 'Sydney' 'SydneyAirport'
 'WaggaWagga' 'Williamtown' 'Wollongong' 'Canberra' 'Tuggeranong'
 'MountGinini' 'Ballarat' 'Bendigo' 'Sale' 'MelbourneAirport' 'Melbourne'
 'Mildura' 'Nhil' 'Portland' 'Watsonia' 'Dartmoor' 'Brisbane' 'Cairns'
 'GoldCoast' 'Townsville' 'Adelaide' 'MountGambier' 'Nuriootpa' 'Woomera'
 'Albany' 'Witchcliffe' 'PearceRAAF' 'PerthAirport' 'Perth' 'SalmonGums'
 'Walpole' 'Hobart' 'Launceston' 'AliceSprings' 'Darwin' 'Katherine'
 'Uluru']
Location  contient  49  labels
['W' 'WNW' 'WSW' 'NE' 'NNW' 'N' 'NNE' 'SW' 'ENE' 'SSE' 'S' 'NW' 'SE' 'ESE'
 nan 'E' 'SSW']
WindGustDir  contient  17  labels
['W' 'NNW' 'SE' 'ENE' 'SW' 'SSE' 'S' 'NE' nan 'SSW' 'N' 'WSW' 'ESE' 'E'
 'NW' 'WNW' 'NNE']
WindDir9am  contient  17  labels
['WNW' 'WSW' 'E' 'NW' 'W'

## 4. Manipulation d'une variable de type date

### Transformer la variable 'Date', qui est codé  comme strings, à une variable 'Date' de type datetime format

In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142193 entries, 0 to 142192
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           142193 non-null  object 
 1   Location       142193 non-null  object 
 2   MinTemp        141556 non-null  float64
 3   MaxTemp        141871 non-null  float64
 4   Rainfall       140787 non-null  float64
 5   Evaporation    81350 non-null   float64
 6   Sunshine       74377 non-null   float64
 7   WindGustDir    132863 non-null  object 
 8   WindGustSpeed  132923 non-null  float64
 9   WindDir9am     132180 non-null  object 
 10  WindDir3pm     138415 non-null  object 
 11  WindSpeed9am   140845 non-null  float64
 12  WindSpeed3pm   139563 non-null  float64
 13  Humidity9am    140419 non-null  float64
 14  Humidity3pm    138583 non-null  float64
 15  Pressure9am    128179 non-null  float64
 16  Pressure3pm    128212 non-null  float64
 17  Cloud9am       88536 non-null

In [44]:

df['Date'] = pd.to_datetime(df['Date'])
df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142193 entries, 0 to 142192
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   Date           142193 non-null  datetime64[ns]
 1   Location       142193 non-null  object        
 2   MinTemp        141556 non-null  float64       
 3   MaxTemp        141871 non-null  float64       
 4   Rainfall       140787 non-null  float64       
 5   Evaporation    81350 non-null   float64       
 6   Sunshine       74377 non-null   float64       
 7   WindGustDir    132863 non-null  object        
 8   WindGustSpeed  132923 non-null  float64       
 9   WindDir9am     132180 non-null  object        
 10  WindDir3pm     138415 non-null  object        
 11  WindSpeed9am   140845 non-null  float64       
 12  WindSpeed3pm   139563 non-null  float64       
 13  Humidity9am    140419 non-null  float64       
 14  Humidity3pm    138583 non-null  float64       
 15  

### Extraire l'année à partir de la variable Date et l'ajouter dans le jeu de données dans une colonne separée 'Years' 

In [16]:


df['Year'] = df['Date'].dt.year
df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow,Year
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No,2008
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No,2008
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No,2008
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No,2008
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No,2008


### Extraire le mois à partir de la variable Date et l'ajouter dans le  jeu de données dans une colonne separée 'Month' 

In [17]:

df['Month'] = df['Date'].dt.month

df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow,Year,Month
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No,2008,12
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No,2008,12
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No,2008,12
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No,2008,12
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No,2008,12


### Extraire le jour à partir de la variable Date et l'ajouter dans le  jeu de données dans une colonne separée 'Day' 

In [18]:
df['Day'] = df['Date'].dt.day

df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow,Year,Month,Day
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,1007.1,8.0,NaN,16.9,21.8,No,No,2008,12,1
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,1007.8,NaN,NaN,17.2,24.3,No,No,2008,12,2
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,1008.7,NaN,2.0,21.0,23.2,No,No,2008,12,3
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,1012.8,NaN,NaN,18.1,26.5,No,No,2008,12,4
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,1006.0,7.0,8.0,17.8,29.7,No,No,2008,12,5


### Supprimer la variable Date du  jeu de données

In [49]:
df.drop('Date', axis=1, inplace = True)
df.head()

,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,WNW,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,WSW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,WSW,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,E,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,NW,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


## 5. Exploration des variables quantitatives

### Afficher la liste des variables quantitatives dans le jeu de données

In [50]:
for var in df.columns:
    
    if df[var].dtype!='O':
        print(var)
    

MinTemp
MaxTemp
Rainfall
Evaporation
Sunshine
WindGustSpeed
WindSpeed9am
WindSpeed3pm
Humidity9am
Humidity3pm
Pressure9am
Pressure3pm
Cloud9am
Cloud3pm
Temp9am
Temp3pm


In [51]:
numerical = [var for var in df.columns if df[var].dtype!='O']

print('Il y a {} variables quantitatives \n'.format(len(numerical)))

print('Les variables quantitatives sont :', numerical)

Il y a 16 variables quantitatives 

Les variables quantitatives sont : ['MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation', 'Sunshine', 'WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm', 'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am', 'Temp3pm']


### Visualiser le contenu des variables quantitatives dans le jeu de données

In [21]:
df[numerical].head()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,Year,Month,Day
0,13.4,22.9,0.6,NaN,NaN,44.0,20.0,24.0,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,2008,12,1
1,7.4,25.1,0.0,NaN,NaN,44.0,4.0,22.0,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,2008,12,2
2,12.9,25.7,0.0,NaN,NaN,46.0,19.0,26.0,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,2008,12,3
3,9.2,28.0,0.0,NaN,NaN,24.0,11.0,9.0,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,2008,12,4
4,17.5,32.3,1.0,NaN,NaN,41.0,7.0,20.0,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,2008,12,5


### Checker les valeurs manquantes dans les variables quantitatives

In [52]:


df[numerical].isna().sum()

MinTemp            637
MaxTemp            322
Rainfall          1406
Evaporation      60843
Sunshine         67816
WindGustSpeed     9270
WindSpeed9am      1348
WindSpeed3pm      2630
Humidity9am       1774
Humidity3pm       3610
Pressure9am      14014
Pressure3pm      13981
Cloud9am         53657
Cloud3pm         57094
Temp9am            904
Temp3pm           2726
dtype: int64

### Afficher le résumé statistique des variables quantitatives (describe())

In [55]:
print(round(df[numerical].describe(),2))

         MinTemp    MaxTemp   Rainfall  Evaporation  Sunshine  WindGustSpeed  \
count  141556.00  141871.00  140787.00     81350.00  74377.00      132923.00   
mean       12.19      23.23       2.35         5.47      7.62          39.98   
std         6.40       7.12       8.47         4.19      3.78          13.59   
min        -8.50      -4.80       0.00         0.00      0.00           6.00   
25%         7.60      17.90       0.00         2.60      4.90          31.00   
50%        12.00      22.60       0.00         4.80      8.50          39.00   
75%        16.80      28.20       0.80         7.40     10.60          48.00   
max        33.90      48.10     371.00       145.00     14.50         135.00   

       WindSpeed9am  WindSpeed3pm  Humidity9am  Humidity3pm  Pressure9am  \
count     140845.00     139563.00    140419.00    138583.00    128179.00   
mean          14.00         18.64        68.84        51.48      1017.65   
std            8.89          8.80        19.05     

## 6. Notre variable target (Y) est le RainTomorrow, Récuprer X et Y à partir du jeu de données 

In [56]:

X = df.drop(['RainTomorrow'], axis=1)

y = df['RainTomorrow']


## 7. Fractionnement le jeu de données en jeu d'entraînement et jeu de test (20% pour le test)

In [61]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

X_train.shape, X_test.shape, y_test.shape



((113754, 21), (28439, 21), (28439,))

In [62]:
X_test

,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,...,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday
79418,Dartmoor,14.3,22.3,0.0,11.6,12.5,SSW,43.0,WSW,SW,...,19.0,69.0,55.0,1013.4,1013.9,NaN,NaN,16.9,20.8,No
93654,Townsville,25.2,33.4,0.0,9.8,NaN,NNW,37.0,N,NE,...,17.0,57.0,49.0,1012.7,1008.8,4.0,NaN,29.5,31.4,No
10042,CoffsHarbour,21.0,27.6,40.4,9.4,6.0,S,50.0,NW,S,...,30.0,85.0,84.0,1006.9,1007.8,7.0,7.0,22.3,23.4,Yes
118260,Perth,18.8,37.3,0.0,12.4,12.8,E,39.0,E,SSW,...,15.0,45.0,24.0,1007.6,1004.2,1.0,1.0,25.2,36.6,No
125564,Walpole,12.2,20.6,0.8,NaN,NaN,NE,22.0,NaN,NNE,...,7.0,99.0,77.0,1014.1,1011.0,NaN,NaN,16.0,18.6,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89574,GoldCoast,20.5,32.0,0.0,NaN,NaN,SE,48.0,NNW,N,...,35.0,49.0,75.0,1009.3,1005.2,NaN,NaN,27.3,24.7,No
55797,Ballarat,7.2,27.4,0.0,NaN,NaN,SSE,35.0,SSW,ENE,...,17.0,54.0,33.0,1013.9,1012.2,8.0,8.0,17.0,23.8,No
132148,Launceston,-1.8,13.1,0.0,NaN,NaN,NW,19.0,E,NW,...,9.0,99.0,69.0,NaN,NaN,NaN,NaN,0.7,12.3,No
80541,Dartmoor,2.5,17.3,0.0,2.0,9.5,E,31.0,NaN,ESE,...,17.0,100.0,51.0,1026.5,1022.4,NaN,NaN,7.8,16.4,No


## 8. Checker les valeurs manquantes des variables quantitatives du jeu d'entraînement et de test

In [26]:
# Récupérer tout d'abord les variables quantitatives (numerical)

numerical = [col for col in X_train.columns if X_train[col].dtypes != 'O']

numerical


# check missing values in numerical variables in X_train
print("Train")
print("*************")
print(X_train[numerical].isnull().sum())

# check missing values in numerical variables in X_test

print("===========================================")
print("Test")
print("*************")
print(X_test[numerical].isnull().sum())



Train
*************
MinTemp            495
MaxTemp            264
Rainfall          1139
Evaporation      48718
Sunshine         54314
WindGustSpeed     7367
WindSpeed9am      1086
WindSpeed3pm      2094
Humidity9am       1449
Humidity3pm       2890
Pressure9am      11212
Pressure3pm      11186
Cloud9am         43137
Cloud3pm         45768
Temp9am            740
Temp3pm           2171
Year                 0
Month                0
Day                  0
dtype: int64
Test
*************
MinTemp            142
MaxTemp             58
Rainfall           267
Evaporation      12125
Sunshine         13502
WindGustSpeed     1903
WindSpeed9am       262
WindSpeed3pm       536
Humidity9am        325
Humidity3pm        720
Pressure9am       2802
Pressure3pm       2795
Cloud9am         10520
Cloud3pm         11326
Temp9am            164
Temp3pm            555
Year                 0
Month                0
Day                  0
dtype: int64


## 9. En utilisant SimpleImputer de Scikit-Learn remplacer les valeurs manquantes dans les variables numérique (entraînement et test) par la moyenne

In [64]:
from sklearn.impute import SimpleImputer

imptr_train=SimpleImputer(missing_values=np.nan, strategy='mean')


X_train[numerical]=imptr_train.fit_transform(X_train[numerical])

imptr_test=SimpleImputer(missing_values=np.nan, strategy='mean')
X_test[numerical]=imptr_test.fit_transform(X_test[numerical])

X_train

C:\Users\ISEN\anaconda3\envs\python37\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\ISEN\anaconda3\envs\python37\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
C:\Users\ISEN\anaconda3\envs\python37\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,...,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday
101982,Nuriootpa,6.1,19.9,0.0,5.478837,7.632372,W,81.0,W,WSW,...,44.0,37.0,70.0,1013.7,1015.1,4.433099,4.501027,18.6,15.8,No
49150,Tuggeranong,14.3,29.0,0.0,5.478837,7.632372,NNW,43.0,WNW,NNW,...,17.0,54.0,39.0,1016.2,1013.2,4.433099,4.501027,20.9,26.1,No
98496,MountGambier,6.1,14.6,0.0,1.000000,3.500000,N,30.0,NNE,NNE,...,13.0,88.0,63.0,1027.9,1024.2,7.000000,7.000000,8.8,13.4,No
113644,PearceRAAF,13.3,17.1,13.4,5.478837,3.900000,SSW,43.0,W,SSW,...,26.0,85.0,62.0,1013.2,1015.5,7.000000,5.000000,17.0,16.0,Yes
127552,Hobart,11.5,19.1,7.0,0.200000,7.100000,ESE,33.0,NNW,ESE,...,24.0,62.0,70.0,1016.8,1015.7,3.000000,6.000000,15.9,18.2,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102373,Nuriootpa,6.7,24.5,0.0,6.500000,12.500000,SE,46.0,ENE,NE,...,9.0,55.0,38.0,1024.2,1020.1,0.000000,3.000000,12.5,22.9,No
119074,Perth,20.1,25.5,0.0,7.400000,0.000000,NW,31.0,NE,NW,...,15.0,25.0,59.0,1012.4,1010.7,8.000000,8.000000,24.1,24.1,No
22073,NorfolkIsland,17.1,24.4,1.0,5.200000,7.200000,E,50.0,ESE,E,...,24.0,75.0,52.0,1020.1,1018.4,7.000000,6.000000,20.5,23.5,No
71306,Nhil,2.3,16.9,0.0,5.478837,7.632372,W,15.0,E,W,...,7.0,91.0,54.0,1024.7,1023.4,4.433099,4.501027,3.8,16.8,No


## 10. Vérifier que les valeurs manquantes n'existent plus dans les variables quantitatives (entraînement et test)

In [28]:
print(X_train[numerical].isnull().sum())
print("======================================")
print(X_test[numerical].isnull().sum())

MinTemp          0
MaxTemp          0
Rainfall         0
Evaporation      0
Sunshine         0
WindGustSpeed    0
WindSpeed9am     0
WindSpeed3pm     0
Humidity9am      0
Humidity3pm      0
Pressure9am      0
Pressure3pm      0
Cloud9am         0
Cloud3pm         0
Temp9am          0
Temp3pm          0
Year             0
Month            0
Day              0
dtype: int64
MinTemp          0
MaxTemp          0
Rainfall         0
Evaporation      0
Sunshine         0
WindGustSpeed    0
WindSpeed9am     0
WindSpeed3pm     0
Humidity9am      0
Humidity3pm      0
Pressure9am      0
Pressure3pm      0
Cloud9am         0
Cloud3pm         0
Temp9am          0
Temp3pm          0
Year             0
Month            0
Day              0
dtype: int64


## 11. En utilisant la fonction fillna() et mode() remplacer les valeurs manquantes dans les variables qualitatives par la valeur la plus fréquante pour chaque variable.

In [66]:

# Récuperer tout d'abord les variables qualitatives

categorical = [col for col in X_train.columns if X_train[col].dtypes == 'O']

categorical





['Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm', 'RainToday']

In [67]:
print(X_train[categorical].isnull().sum())
print("========================")
print(X_test[categorical].isnull().sum())

Location          0
WindGustDir    7416
WindDir9am     8039
WindDir3pm     3028
RainToday      1111
dtype: int64
Location          0
WindGustDir    1914
WindDir9am     1974
WindDir3pm      750
RainToday       295
dtype: int64


In [73]:
print(X_train['WindGustDir'].mode()[0])
print(X_train['WindDir9am'].mode()[0])
print(X_train['RainToday'].mode()[0])

W
N
No


In [75]:


for df2 in [X_train, X_test]:
    df2['WindGustDir'].fillna(X_train['WindGustDir'].mode()[0], inplace=True)
    df2['WindDir9am'].fillna(X_train['WindDir9am'].mode()[0], inplace=True)
    df2['WindDir3pm'].fillna(X_train['WindDir3pm'].mode()[0], inplace=True)
    df2['RainToday'].fillna(X_train['RainToday'].mode()[0], inplace=True)
    

C:\Users\ISEN\anaconda3\envs\python37\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


## 12. Vérifier que les valeurs manquantes n'existent plus dans les variables qualitative (entraînement et test)

In [76]:
X_train[categorical].isnull().sum()
X_test[categorical].isnull().sum()

Location       0
WindGustDir    0
WindDir9am     0
WindDir3pm     0
RainToday      0
dtype: int64

## 13. Encoder la variable qualitative 'RainToday' en utilisant le module category_encoders et la fonction BinaryEncoder(cols=['RainToday']) pour le jeu d'entraînement et de test.

In [77]:
X_train.head()

,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,...,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday
101982,Nuriootpa,6.1,19.9,0.0,5.478837,7.632372,W,81.0,W,WSW,...,44.0,37.0,70.0,1013.7,1015.1,4.433099,4.501027,18.6,15.8,No
49150,Tuggeranong,14.3,29.0,0.0,5.478837,7.632372,NNW,43.0,WNW,NNW,...,17.0,54.0,39.0,1016.2,1013.2,4.433099,4.501027,20.9,26.1,No
98496,MountGambier,6.1,14.6,0.0,1.000000,3.500000,N,30.0,NNE,NNE,...,13.0,88.0,63.0,1027.9,1024.2,7.000000,7.000000,8.8,13.4,No
113644,PearceRAAF,13.3,17.1,13.4,5.478837,3.900000,SSW,43.0,W,SSW,...,26.0,85.0,62.0,1013.2,1015.5,7.000000,5.000000,17.0,16.0,Yes
127552,Hobart,11.5,19.1,7.0,0.200000,7.100000,ESE,33.0,NNW,ESE,...,24.0,62.0,70.0,1016.8,1015.7,3.000000,6.000000,15.9,18.2,Yes


In [85]:
# Installation: Dans Anaconda Prompt tapez la commande: pip install category_encoders

import category_encoders as ce

encoder = ce.BinaryEncoder(cols=['RainToday'])

X_train = encoder.fit_transform(X_train)

X_test = encoder.transform(X_test)

X_train.head()

,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday_0,RainToday_1
101982,Nuriootpa,6.1,19.9,0.0,5.478837,7.632372,W,81.0,W,WSW,...,37.0,70.0,1013.7,1015.1,4.433099,4.501027,18.6,15.8,0,1
49150,Tuggeranong,14.3,29.0,0.0,5.478837,7.632372,NNW,43.0,WNW,NNW,...,54.0,39.0,1016.2,1013.2,4.433099,4.501027,20.9,26.1,0,1
98496,MountGambier,6.1,14.6,0.0,1.000000,3.500000,N,30.0,NNE,NNE,...,88.0,63.0,1027.9,1024.2,7.000000,7.000000,8.8,13.4,0,1
113644,PearceRAAF,13.3,17.1,13.4,5.478837,3.900000,SSW,43.0,W,SSW,...,85.0,62.0,1013.2,1015.5,7.000000,5.000000,17.0,16.0,1,0
127552,Hobart,11.5,19.1,7.0,0.200000,7.100000,ESE,33.0,NNW,ESE,...,62.0,70.0,1016.8,1015.7,3.000000,6.000000,15.9,18.2,1,0


## 14. Recréer un nouveau jeu d'entrainement en concatenant les variables numériques du jeu d'entrainement précedent avec les deux colonnes RainToday_0', 'RainToday_1'qui vont se créer à l'aide de l'étape précedente, plus les variables qualitatives transformées à des variables muettes avec la fonction get_dummies() de pandas 

In [86]:
X_train.WindGustDir

101982      W
49150     NNW
98496       N
113644    SSW
127552    ESE
         ... 
102373     SE
119074     NW
22073       E
71306       W
77135       S
Name: WindGustDir, Length: 113754, dtype: object

In [87]:
pd.get_dummies(X_train.WindGustDir)

,E,ENE,ESE,N,NE,NNE,NNW,NW,S,SE,SSE,SSW,SW,W,WNW,WSW
101982,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
49150,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
98496,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
113644,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
127552,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102373,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
119074,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
22073,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
71306,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [88]:
X_train = pd.concat([X_train[numerical], X_train[['RainToday_0', 'RainToday_1']],
                     pd.get_dummies(X_train.Location), 
                     pd.get_dummies(X_train.WindGustDir),
                     pd.get_dummies(X_train.WindDir9am),
                     pd.get_dummies(X_train.WindDir3pm)], axis=1)

print(X_train.shape)

(113754, 115)


In [89]:
X_train.head()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,...,NNW,NW,S,SE,SSE,SSW,SW,W,WNW,WSW
101982,6.1,19.9,0.0,5.478837,7.632372,81.0,48.0,44.0,37.0,70.0,...,0,0,0,0,0,0,0,0,0,1
49150,14.3,29.0,0.0,5.478837,7.632372,43.0,2.0,17.0,54.0,39.0,...,1,0,0,0,0,0,0,0,0,0
98496,6.1,14.6,0.0,1.000000,3.500000,30.0,11.0,13.0,88.0,63.0,...,0,0,0,0,0,0,0,0,0,0
113644,13.3,17.1,13.4,5.478837,3.900000,43.0,15.0,26.0,85.0,62.0,...,0,0,0,0,0,1,0,0,0,0
127552,11.5,19.1,7.0,0.200000,7.100000,33.0,13.0,24.0,62.0,70.0,...,0,0,0,0,0,0,0,0,0,0


## 15. Refaire l'étape précedente pour le jeu de test

In [90]:
X_test = pd.concat([X_test[numerical], X_test[['RainToday_0', 'RainToday_1']],
                     pd.get_dummies(X_test.Location), 
                     pd.get_dummies(X_test.WindGustDir),
                     pd.get_dummies(X_test.WindDir9am),
                     pd.get_dummies(X_test.WindDir3pm)], axis=1)

print(X_test.shape)

(28439, 115)


## 16. Standariser votre jeu d'entrainement et de test à l'aide de StandardScaler

In [91]:
cols = X_train.columns
cols

Index(['MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation', 'Sunshine',
       'WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am',
       'Humidity3pm',
       ...
       'NNW', 'NW', 'S', 'SE', 'SSE', 'SSW', 'SW', 'W', 'WNW', 'WSW'],
      dtype='object', length=115)

In [92]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)



In [93]:
X_test = scaler.fit_transform(X_test)

In [94]:
X_test

array([[ 0.33319978, -0.12346289, -0.27693627, ..., -0.27288694,
        -0.25356262, -0.26186434],
       [ 2.03793445,  1.43638711, -0.27693627, ..., -0.27288694,
        -0.25356262, -0.26186434],
       [ 1.38106421,  0.62133036,  4.57642784, ..., -0.27288694,
        -0.25356262, -0.26186434],
       ...,
       [-2.18480281, -1.41631154, -0.27693627, ..., -0.27288694,
        -0.25356262, -0.26186434],
       [-1.5122928 , -0.82609803, -0.27693627, ..., -0.27288694,
        -0.25356262, -0.26186434],
       [-1.21513721,  0.56511954, -0.27693627, ..., -0.27288694,
        -0.25356262, -0.26186434]])

In [95]:
X_train = pd.DataFrame(X_train, columns=[cols])

X_test = pd.DataFrame(X_test, columns=[cols])

In [96]:
X_train

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,...,NNW,NW,S,SE,SSE,SSW,SW,W,WNW,WSW
0,-0.953502,-0.469740,-0.279501,0.000000,0.000000,3.125950,3.844707,2.906482,-1.675725,0.903447,...,-0.240443,-0.250032,-0.26949,-0.336039,-0.261176,-0.243415,-0.263702,-0.273929,-0.254858,3.762860
1,0.330229,0.810536,-0.279501,0.000000,0.000000,0.229621,-1.355442,-0.187086,-0.779375,-0.605628,...,4.158996,-0.250032,-0.26949,-0.336039,-0.261176,-0.243415,-0.263702,-0.273929,-0.254858,-0.265755
2,-0.953502,-1.215395,-0.279501,-1.408176,-1.511885,-0.761229,-0.338021,-0.645392,1.013326,0.562688,...,-0.240443,-0.250032,-0.26949,-0.336039,-0.261176,-0.243415,-0.263702,-0.273929,-0.254858,-0.265755
3,0.173677,-0.863671,1.306723,0.000000,-1.365540,0.229621,0.114165,0.844103,0.855146,0.514008,...,-0.240443,-0.250032,-0.26949,-0.336039,-0.261176,4.108213,-0.263702,-0.273929,-0.254858,-0.265755
4,-0.108118,-0.582292,0.549124,-1.659702,-0.194776,-0.532571,-0.111928,0.614950,-0.357563,0.903447,...,-0.240443,-0.250032,-0.26949,-0.336039,-0.261176,-0.243415,-0.263702,-0.273929,-0.254858,-0.265755
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113749,-0.859570,0.177433,-0.279501,0.321060,1.780889,0.458278,0.340259,-1.103699,-0.726648,-0.654308,...,-0.240443,-0.250032,-0.26949,-0.336039,-0.261176,-0.243415,-0.263702,-0.273929,-0.254858,-0.265755
113750,1.238234,0.318122,-0.279501,0.604026,-2.792408,-0.685010,-0.564115,-0.416239,-2.308443,0.367969,...,-0.240443,3.999496,-0.26949,-0.336039,-0.261176,-0.243415,-0.263702,-0.273929,-0.254858,-0.265755
113751,0.768577,0.163364,-0.161126,-0.087668,-0.158189,0.763155,0.340259,0.614950,0.327881,0.027210,...,-0.240443,-0.250032,-0.26949,-0.336039,-0.261176,-0.243415,-0.263702,-0.273929,-0.254858,-0.265755
113752,-1.548402,-0.891809,-0.279501,0.000000,0.000000,-1.904517,-0.903255,-1.332852,1.171505,0.124570,...,-0.240443,-0.250032,-0.26949,-0.336039,-0.261176,-0.243415,-0.263702,3.650575,-0.254858,-0.265755


## 17. Créer votre modèle de régression logistique et entraînez le sur les données d'entraînement

In [97]:
# train a logistic regression model on the training set
from sklearn.linear_model import LogisticRegression


# instantiate the model
logreg = LogisticRegression(solver='liblinear', random_state=0)

print(y_train.shape)
# fit the model
logreg.fit(X_train, y_train)

(113754,)


LogisticRegression(random_state=0, solver='liblinear')

## 18. Predire les résultats du modèle sur l'ensemble du test

In [98]:
y_pred_test = logreg.predict(X_test)

print(y_pred_test)
print(y_test)


['No' 'No' 'Yes' ... 'No' 'No' 'No']
79418      No
93654      No
10042      No
118260     No
125564     No
         ... 
89574     Yes
55797      No
132148     No
80541      No
123750    Yes
Name: RainTomorrow, Length: 28439, dtype: object


### Calculer la probabilité que demain va pleuvoir ou pas en utilisant la fonction predict_proba sur le jeu de test

In [45]:
logreg.predict_proba(X_test)

array([[0.91405569, 0.08594431],
       [0.86150045, 0.13849955],
       [0.83929301, 0.16070699],
       ...,
       [0.97825507, 0.02174493],
       [0.80204587, 0.19795413],
       [0.33255711, 0.66744289]])

In [46]:
# probability of getting output as 0 - no rain

print(logreg.predict_proba(X_test)[:,0])

# probability of getting output as 1 - rain

print(logreg.predict_proba(X_test)[:,1])

[0.91405569 0.86150045 0.83929301 ... 0.97825507 0.80204587 0.33255711]
[0.08594431 0.13849955 0.16070699 ... 0.02174493 0.19795413 0.66744289]


### Afficher l'accuracy score en utilisant la fonction accuracy_score sur le jeu de test

In [73]:
from sklearn.metrics import accuracy_score

print('Model accuracy score: ',accuracy_score(y_test, y_pred_test))

Model accuracy score:  0.850170540455009


### Comparer l'accuracy du modèle sur l'ensemble d'apprentissage et l'ensemble de test, qu'est ce que vous remarquez ?


In [74]:

# print the scores on training and test set

print('Training set score: ',logreg.score(X_train, y_train))

print('Test set score:', logreg.score(X_test, y_test))

Training set score:  0.8481723719605464
Test set score: 0.850170540455009


##  19. Afficher la matrice de confusion, TP, TN, FP, FN sur le jeu du test en utilisant la fonction confusion_matrix


In [49]:

# Print the Confusion Matrix and slice it into four pieces

from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred_test)

print('Confusion matrix\n\n', cm)

print('\nTrue Positives(TP) = ', cm[0,0])

print('\nTrue Negatives(TN) = ', cm[1,1])

print('\nFalse Positives(FP) = ', cm[0,1])

print('\nFalse Negatives(FN) = ', cm[1,0])

Confusion matrix

 [[20882  1185]
 [ 3076  3296]]

True Positives(TP) =  20882

True Negatives(TN) =  3296

False Positives(FP) =  1185

False Negatives(FN) =  3076


##  20. Calculer les métriques de classification (accuracy, classification error, precision, recall, specificity) en utilisant seulement les valeurs de TP, TN, FP, FN.

In [72]:
TP = cm[0,0]
TN = cm[1,1]
FP = cm[0,1]
FN = cm[1,0]

# print classification accuracy

classification_accuracy = (TP + TN) / float(TP + TN + FP + FN)

print('Classification accuracy :',classification_accuracy)



# print classification error

classification_error = (FP + FN) / float(TP + TN + FP + FN)

print('Classification error : ',classification_error)



# print precision score

precision = TP / float(TP + FP)


print('Precision : ',precision)

# print recall score

recall = TP / float(TP + FN)

print('Recall or Sensitivity :', recall)

# print Specificity score

specificity = TN / (TN + FP)

print('Specificity : ', specificity)

f1= (2*recall*precision)/float(recall+precision)

print('F1 score:', f1)

Classification accuracy : 0.850170540455009
Classification error :  0.14982945954499102
Precision :  0.9462999048352744
Recall or Sensitivity : 0.8716086484681526
Specificity :  0.7355501004240125
F1 score: 0.9074198804997283
